# Make the master table for input to modeling
Load the tables, do some transformation and then join together

In [1]:
import pandas as pd

## Load data

In [2]:
data_input = "data/input/"
companies = pd.read_csv(data_input + "companies.csv")
reviews = pd.read_csv(data_input + "reviews.csv")
shuttles = pd.read_excel(data_input + "shuttles.xlsx")

## Helper functions

In [3]:
def _is_true(x):
    return x == "t"

def _parse_percentage(x):
    x = x.str.replace("%", "")
    x = x.astype(float) / 100
    return x

def _parse_money(x):
    x = x.str.replace("$", '').str.replace(",", '')
    x = x.astype(float)
    return x

## Main functions to run

In [4]:
def preprocess_companies(companies):
    companies['iata_approved'] = _is_true(companies['iata_approved'])
    companies["company_rating"] = _parse_percentage(companies['company_rating'])
    return companies

In [5]:
def preprocess_shuttles(shuttles):
    shuttles["d_check_complete"] = _is_true(shuttles["d_check_complete"])
    shuttles['moon_clearance_complete'] = _is_true(shuttles["moon_clearance_complete"])
    shuttles["price"] = _parse_money(shuttles['price'])
    return shuttles

In [6]:
def create_master_table(shuttles, companies, reviews):
    rated_shuttles = shuttles.merge(reviews, left_on="id", right_on="shuttle_id")
    master_table = rated_shuttles.merge(companies, left_on="company_id", right_on="id")
    master_table = master_table.dropna()
    return master_table

## Run

In [7]:
processed_companies = preprocess_companies(companies)
processed_shuttles = preprocess_shuttles(shuttles)
master_table = create_master_table(shuttles, companies, reviews)

## Save data

In [8]:
master_table.to_csv("data/intermediate/master_table.csv")